In [2]:
import { TextLoader } from "langchain/document_loaders/fs/text";
import { RecursiveCharacterTextSplitter } from "langchain/text_splitter";
const loader = new TextLoader("data/kong.txt");
const docs = await loader.load();

const splitter = new RecursiveCharacterTextSplitter({
    chunkSize: 100,
    chunkOverlap: 20,
  });

const splitDocs = await splitter.splitDocuments(docs);
splitDocs

[
  Document {
    pageContent: "鲁镇的酒店的格局，是和别处不同的：都是当街一个曲尺形的大柜台，柜里面预备着热水，可以随时温酒。做工的人，傍午傍晚散了工，每每花四文铜钱，买一碗酒，——这是二十多年前的事，现在每碗要涨到十文，——靠柜",
    metadata: { source: "data/kong.txt", loc: { lines: { from: 2, to: 2 } } },
    id: undefined
  },
  Document {
    pageContent: "多年前的事，现在每碗要涨到十文，——靠柜外站着，热热的喝了休息；倘肯多花一文，便可以买一碟盐煮笋，或者茴香豆，做下酒物了，如果出到十几文，那就能买一样荤菜，但这些顾客，多是短衣帮⑴，大抵没有这样阔绰⑵",
    metadata: { source: "data/kong.txt", loc: { lines: { from: 2, to: 2 } } },
    id: undefined
  },
  Document {
    pageContent: "些顾客，多是短衣帮⑴，大抵没有这样阔绰⑵。只有穿长衫的，才踱进店面隔壁的房子里，要酒要菜，慢慢地坐喝。",
    metadata: { source: "data/kong.txt", loc: { lines: { from: 2, to: 2 } } },
    id: undefined
  },
  Document {
    pageContent: "我从十二岁起，便在镇口的咸亨酒店里当伙计，掌柜说，样子太傻，怕侍候不了长衫主顾，就在外面做点事罢。外面的短衣主顾，虽然容易说话，但唠唠叨叨缠夹不清的也很不少。他们往往要亲眼看着黄酒从坛子里舀出，看过",
    metadata: { source: "data/kong.txt", loc: { lines: { from: 3, to: 3 } } },
    id: undefined
  },
  Document {
    pageContent: "他们往往要亲眼看着黄酒从坛子里舀出，看过壶子底里有水没有，又亲看将壶子放在热水里，然后放心：在这严重监督下，羼⑶水也很为难。所以过了几天，掌柜又说我

In [1]:
//deno-ignore-cell
import { OpenAIEmbeddings } from "@langchain/openai";
const embeddings = new OpenAIEmbeddings()
console.log(splitDocs[0])
// Document {
//   pageContent: "鲁镇的酒店的格局，是和别处不同的：都是当街一个曲尺形的大柜台，柜里面预备着热水，可以随时温酒。做工的人，傍午傍晚散了工，每每花四文铜钱，买一碗酒，——这是二十多年前的事，现在每碗要涨到十文，——靠柜外",
//   metadata: { source: "data/kong.txt", loc: { lines: { from: 1, to: 1 } } }
// }


ReferenceError: splitDocs is not defined

bge-large和bge-m3都是智谱开源的embeding模型  
large是中文专用模型，适合纯中文场景，并且模型1.3B较大，资源消耗高  
m3是多语言模型，资源消耗更低  


In [ ]:
import { OllamaEmbeddings } from "npm:@langchain/ollama";
const embeddings = new OllamaEmbeddings({
  model: "bge-m3",
  baseUrl: "http://localhost:11434", // Default value
});
const res =await embeddings.embedQuery(splitDocs[0].pageContent)
res

[
    0.008740378,    0.04719728,  -0.033101488,   0.026490076, -0.016458334,
   -0.007930168, -0.0060368213,   0.038651697,   0.027293922, -0.029894743,
    0.014001312,   0.013423827,  -0.049799044, -0.0127271665,  0.055749197,
   0.0030935942,   0.029488508,  -0.009582613, 0.00053053827, -0.038016077,
   -0.008396971,   0.009748647,  -0.023752507,   0.056000393, 0.0004296581,
   0.0034583604,  -0.037966553,   0.028785648,   0.066896304,  0.036451742,
    0.016272554,  -0.010934695,   0.026729824,  -0.014664751,  0.040016092,
   -0.017349761,  -0.029412014,  -0.007202131,  -0.026093092,    0.0738953,
    0.009295809,   0.001959285,    0.02944274,   0.020640664, -0.031330626,
    -0.06177479,  -0.012812906,  -0.023647763,   -0.03839418, -0.008633445,
    -0.02122242,   0.004130641,   0.044303004,  -0.024918107, -0.052432578,
    0.029094325, -0.0005865712,  -0.004050862,  -0.046737757,   0.02026808,
    -0.02235496, -0.0041085132,    0.01386025,  -0.007281756, 0.0027739669,
    0.0620

## 创建 MemoryVectorStore

这是在内存中构建的向量数据库  
注意，因为 embedding 向量是需要有一定的花费的，所以仅在学习和测试时使用 MemoryVectorStore，而在真实项目中，搭建其他向量数据库，或者使用云数据库。  


In [ ]:
import { MemoryVectorStore } from "langchain/vectorstores/memory";

const vectorstore = new MemoryVectorStore(embeddings);
await vectorstore.addDocuments(splitDocs);


In [ ]:
// 创建一个 retriever，这也是可以直接从 vector store 的实例中自动生成，这里我们传入了参数 2，代表对应每个输入，我们想要返回相似度最高的两个文本内容
const retriever = vectorstore.asRetriever(2)
// 然后，我们就可以使用 retriever 来进行文档的提取
await retriever.invoke("茴香豆是做什么用的")
// 可以看到结果提取出茴香豆相关的内容

[
  Document {
    pageContent: "你读过书么？”我略略点一点头。他说，“读过书，……我便考你一考。茴香豆的茴字，怎样写的？”我想，讨饭一样的人，也配考我么？便回过脸去，不再理会。孔乙己等了许久，很恳切的说道，“不能写罢？……我教给你，",
    metadata: { source: "data/kong.txt", loc: { lines: { from: 8, to: 8 } } },
    id: undefined
  },
  Document {
    pageContent: "很恳切的说道，“不能写罢？……我教给你，记着！这些字应该记着。将来做掌柜的时候，写账要用。”我暗想我和掌柜的等级还很远呢，而且我们掌柜也从不将茴香豆上账；又好笑，又不耐烦，懒懒的答他道，“谁要你教，不",
    metadata: { source: "data/kong.txt", loc: { lines: { from: 8, to: 8 } } },
    id: undefined
  }
]

In [ ]:
await retriever.invoke("下酒菜一般是什么？")
// 提取的时候，是根据相似度进行度量的，所以如果用户提问的特别简洁，并没有相应的关键词，就会出现提取的信息错误的问题，
// 下面没有截取到孔乙己下酒物相关的（温两碗酒，要一碟茴香豆）分块

[
  Document {
    pageContent: "多年前的事，现在每碗要涨到十文，——靠柜外站着，热热的喝了休息；倘肯多花一文，便可以买一碟盐煮笋，或者茴香豆，做下酒物了，如果出到十几文，那就能买一样荤菜，但这些顾客，多是短衣帮⑴，大抵没有这样阔绰⑵",
    metadata: { source: "data/kong.txt", loc: { lines: { from: 2, to: 2 } } },
    id: undefined
  },
  Document {
    pageContent: "些顾客，多是短衣帮⑴，大抵没有这样阔绰⑵。只有穿长衫的，才踱进店面隔壁的房子里，要酒要菜，慢慢地坐喝。",
    metadata: { source: "data/kong.txt", loc: { lines: { from: 2, to: 2 } } },
    id: undefined
  }
]

In [ ]:
// 如果涉及多层语意的情况，也会有问题，有运气的成分，所以返回更多的数据源就会有价值
await retriever.invoke("孔乙己用什么谋生？")

[
  Document {
    pageContent: "孔乙己是这样的使人快活，可是没有他，别人也便这么过。",
    metadata: { source: "data/kong.txt", loc: { lines: { from: 10, to: 10 } } },
    id: undefined
  },
  Document {
    pageContent: "听人家背地里谈论，孔乙己原来也读过书，但终于没有进学⑼，又不会营生⑽；于是愈过愈穷，弄到将要讨饭了。幸而写得一笔好字，便替人家钞⑾钞书，换一碗饭吃。可惜他又有一样坏脾气，便是好喝懒做。坐不到几天，便",
    metadata: { source: "data/kong.txt", loc: { lines: { from: 6, to: 6 } } },
    id: undefined
  }
]

## 构建本地vector store
这里我们使用faiss，这是facebook开源的，node和python都可以使用

也可以使用HNSWLib  

另外faiss-node，使用pnpm直接安装，langchain里面会报导入错误。。。


Faiss是轻量级库，适合小规模数据（百万级以下）
python中可以用Milvus（十亿级）


选型决策树
根据需求快速匹配最合适的工具：

1. 开发阶段验证或小型项目 → ChromaDB（简单、无依赖）。
2. 生产环境且需全托管服务 → Pinecone（省心）或 Zilliz Cloud（超大规模）。
3. 已有 PostgreSQL 数据库 → PGVector + pgvector（无需引入新组件）。
4. 需要极致性能的本地实验 → FAISS（内存索引，无需网络延迟）。
5. 企业级海量数据+复杂查询 → Milvus（自建）或 Weaviate（混合搜索）。
